# [Module 4.1.2] inference on model from Kubeflow


---
이 노트북은 약 10정도 시간이 소요 됩니다.

In [10]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## SageMaker Train Model from KubeFlow

In [22]:
model_data = 's3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-02-01-14-52-546/model/TrainingJob-20200802135153-A952/output/model.tar.gz'
print('sagemaker train job from KubeFlow: \n', model_data)

sagemaker train job from KubeFlow: 
 s3://sagemaker-ap-northeast-2-343441690612/sagemaker-scikit-learn-2020-08-02-01-14-52-546/model/TrainingJob-20200802135153-A952/output/model.tar.gz


In [18]:
inference_image = '343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu'
print("inference_image: ", inference_image)

inference_image:  343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## 엔드포인트 생성

In [21]:
%%time

from sagemaker.tensorflow.serving import Model

model = Model(model_data= model_data,
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


---------------!CPU times: user 1min 4s, sys: 11.2 s, total: 1min 15s
Wall time: 8min 54s


## Predictor Creation on the Endpoint

In [29]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-08-02-14-33-21-637


In [30]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [44]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [45]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
8104,if high school musical was fifty shades of grey,7
4788,my whole check is gone i haven't even got pai...,8
8541,just better be upset,8
8914,thats why i fired yo bitch ass at auntie annie's,8
39,jumping at camera hates his pic took staffies...,0
5785,i love you you made my day thank you,0
4660,stomach killing me,7
4093,jewelcandle a jewel in every candle mondaymoti...,1
7268,lost my homework from work because mfs wanna t...,8
8513,i'm not check bio,3


In [46]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 5 이모티콘 추천

In [48]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    tweets = [tweet]    
    predicted_classes = predictor.predict(tweets)[0]
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),   
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        ))    

tweet:  if high school musical was fifty shades of grey 
Ground_truth- 7:😭
 
Prediction: 5,😍,7,😭,0,❤,2,🔥,4,😊 
 
tweet:  my whole check is gone i haven't even got paid yet 
Ground_truth- 8:🙄
 
Prediction: 4,😊,8,🙄,7,😭,0,❤,9,🤔 
 
tweet:  just better be upset 
Ground_truth- 8:🙄
 
Prediction: 6,😩,8,🙄,9,🤔,3,😂,7,😭 
 
tweet: thats why i fired yo bitch ass at auntie annie's 
Ground_truth- 8:🙄
 
Prediction: 8,🙄,9,🤔,7,😭,2,🔥,4,😊 
 
tweet:  jumping at camera hates his pic took staffiesaturday fishface 
Ground_truth- 0:❤
 
Prediction: 7,😭,8,🙄,9,🤔,4,😊,3,😂 
 
tweet:  i love you you made my day thank you  
Ground_truth- 0:❤
 
Prediction: 1,💕,0,❤,4,😊,5,😍,9,🤔 
 
tweet: stomach killing me 
Ground_truth- 7:😭
 
Prediction: 7,😭,5,😍,3,😂,0,❤,8,🙄 
 
tweet: jewelcandle a jewel in every candle mondaymotivation firstdayofschool 
Ground_truth- 1:💕
 
Prediction: 4,😊,5,😍,0,❤,1,💕,2,🔥 
 
tweet: lost my homework from work because mfs wanna touch it and then put it somewhere it wasn't at 
Ground_truth- 8:🙄
 
Prediction: 